In [1]:
import numpy as np
import networkx as nx
import scipy
from karateclub.graph_embedding import WaveletCharacteristic
import torch
from qtensor import QAOA_energy

/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-25 12:03:33,749- WARNING•	QuickBB solver is unavailable
2023-10-25 12:03:33,750- WARNING•	Tamaki solver is unavailable: No path /home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/thirdparty/tamaki_treewidth
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refa

In [2]:
def matrices_to_graphs(matrix_list):
    '''
    This function takes a list of adjacency matrices and generates a list of graphs.
    '''

    g_list = []
    for matrix in matrix_list:
        array = np.array(matrix)
        g = nx.from_numpy_array(array)
        g_list.append(g)

    return g_list

In [3]:
training_file = open("1045_40_node_random_graphs.txt")
training_matrix_list = np.loadtxt(training_file).reshape(1045, 40, 40)
training_graph_list = matrices_to_graphs(training_matrix_list)

In [4]:
test_file = open("graphs/graphs/104_50_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(104, 50, 50)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [5]:
model = WaveletCharacteristic()
model.fit(training_graph_list)
model_array = model.get_embedding()

/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/waveletcharacteristic.py:49: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph, nodelist=range(graph.number_of_nodes()))
<class 'networkx.utils.decorators.argmap'> compilation 12:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/waveletcharacteristic.py:49: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph, nodelist=range(graph.number_of_nodes()))
<class 'networkx.utils.decorators.argmap'> compilation 12:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/

In [83]:
def find_indices(vector):
    '''
    This function returns the indices from the first three closest and last three furthest graphs
    in embedded space to the test graph. The first index in the array corresponds to the graph that
    is most similar, while the last index corresponds to the most dissimilar.
    '''
    
    length = len(vector)
    sorted_vector = sorted(vector)
    
    max_value = sorted_vector[length - 1]
    second_max_value = sorted_vector[length - 2]
    third_max_value = sorted_vector[length - 3]
    
    min_value = sorted_vector[0]
    second_min_value = sorted_vector[1]
    third_min_value = sorted_vector[2]
    
    for i in range(len(vector)):
        if vector[i] == max_value:
            max_index = vector.index(vector[i])
        if vector[i] == second_max_value:
            second_max_index = vector.index(vector[i])
        if vector[i] == third_max_value:
            third_max_index = vector.index(vector[i])
        if vector[i] == min_value:
            min_index = vector.index(vector[i])
        if vector[i] == second_min_value:
            second_min_index = vector.index(vector[i])
        if vector[i] == third_min_value:
            third_min_index = vector.index(vector[i])
        else:
            continue
            
    indices = [min_index, second_min_index, third_min_index, third_max_index, second_max_index, max_index]
    
    return indices

In [120]:
indices = []

for i in range(len(test_graph_list)):
    infer_vector = model.infer([test_graph_list[i]])
    pairwise_distances = []
    for j in range(len(model_array)):
        dist = np.linalg.norm(model_array[j] - infer_vector)
        pairwise_distances.append(dist)
    index = find_indices(pairwise_distances)
    indices.append(index)

/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/waveletcharacteristic.py:49: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph, nodelist=range(graph.number_of_nodes()))
<class 'networkx.utils.decorators.argmap'> compilation 12:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/waveletcharacteristic.py:49: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph, nodelist=range(graph.number_of_nodes()))
<class 'networkx.utils.decorators.argmap'> compilation 12:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/

In [9]:
s = 0

for i in range(len(test_graph_list)):
    if indices[i][0] == i:
        s += 1

In [121]:
new_indices = [indices[0], indices[51], indices[103]]

In [86]:
gamma_params_file = open('graph_data/training_set_optimal_gammas.txt')
gamma_params = np.loadtxt(gamma_params_file).reshape(1045, 20, 3)

beta_params_file = open('graph_data/training_set_optimal_betas.txt')
beta_params = np.loadtxt(beta_params_file).reshape(1045, 20, 3)

In [10]:
test_file = open("graphs/graphs/104_50_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(104, 50, 50)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [123]:
graph_list = [test_graph_list[0], test_graph_list[51], test_graph_list[103]]

In [ ]:
index = 6 # 3 top indices 3 bottom indices
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(len(test_graph_list))]

for i in range(len(test_graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(test_graph_list[i], gamma=gamma_params[indices[i][j]][k], beta=beta_params[indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy

Edge iteration:  52%|█████▎    | 21/40 [00:00<00:00, 42.36it/s, Treewidth=11]

In [124]:
index = 6 # 3 top indices 3 bottom indices
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(len(graph_list))]

for i in range(len(graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(graph_list[i], gamma=gamma_params[new_indices[i][j]][k], beta=beta_params[new_indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy

Edge iteration: 100%|██████████| 74/74 [00:07<00:00,  9.42it/s, Treewidth=13]


In [117]:
index = 6 # 3 top indices 3 bottom indices
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(len(graph_list))]

for i in range(len(graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(graph_list[i], gamma=gamma_params[new_indices[i][j]][k], beta=beta_params[new_indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy

Edge iteration: 100%|██████████| 88/88 [00:04<00:00, 19.91it/s, Treewidth=16]


In [125]:
average_transfer_energies

[[42.4808615401698,
  42.49400771052945,
  42.395712341291535,
  30.874998438360034,
  24.849222158918,
  27.510439346533655],
 [45.999456498638,
  46.767202940831304,
  46.19839037090523,
  39.97240234579259,
  40.59571288814234,
  39.87881393797468],
 [56.102775608505866,
  55.81472603261943,
  55.572150599405425,
  32.46642137044952,
  47.940913100318696,
  35.876347102704436]]